<a href="https://colab.research.google.com/github/bryandaetz1/SB_County_COVID-19_Data/blob/master/SB_County_Public_Health_Web_Scraper_7_18_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
#specify URL/web page to scrape
url = 'https://publichealthsbc.org/status-reports/'

#create user-agent (Windows 10 with Google Chrome)
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '\
'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.03987.149 '\
'Safara/537.36'

headers = {'User-Agent': user_agent}

response = requests.get(url, headers = headers).text

In [3]:
#parse the HTML from the URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(response, 'html.parser')

In [4]:
#grabbing page title to verify that we were able to successfully get requests from webpage 
soup.title

<title>Status Reports – County of Santa Barbara</title>

In [5]:
#after inspecting the page, it looks like all the tables that contain data we want have id = 'stats'
tables = soup.find_all('div', {'class': 'elementor-text-editor elementor-clearfix'})
len(tables)

8

In [6]:
#trying to create function to convert html table to pandas dataframe, had to break this into two functions

def TableToList(table):
  #getting contents of table and converting the text into a list
  raw = list(table.find_all('td'))
  text = []
  for i in raw:
    text.append(i.text)
  return text

def ListToDf(list, no_columns):
  # calculating number of rows for dataframe
  rows = int(len(list) / no_columns)

  # converting list to numpy array to reshape
  array = np.array(list)
  array = np.reshape(array, (rows, no_columns))

  #converting array to pandas dataframe
  df = pd.DataFrame(array[1:], columns = array[0])
  return df


In [7]:
table1 = TableToList(tables[0])
#table1

In [8]:
current_stats = ListToDf(table1, 6)
current_stats

,Geographic Area as of 7/17/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths
0,SOUTH COUNTY UNINCORPORATED AREA includes comm...,3,88,81,6,0
1,CITY OF SANTA BARBARA and the unincorporated a...,19,562,509,51,3
2,CITY OF GOLETA,4,101,89,10,2
3,COMMUNITY OF ISLA VISTA,0,29,28,1,0
4,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,3,77,71,5,1
5,SANTA YNEZ VALLEY including the Cities of Solv...,2,51,41,10,0
6,CITY OF LOMPOC and the communities of Mission ...,18,288,241,43,4
7,FEDERAL PRISON IN LOMPOC,6,1007,997,7,3
8,CITY OF SANTA MARIA,69,2099,1862,220,18
9,COMMUNITY OF ORCUTT,5,143,131,11,0


In [9]:
table2 = TableToList(tables[1])
table2 = table2[1:]
#table2

In [10]:
cases_by_age = ListToDf(table2, 5)
cases_by_age

,Age Range,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,0 – 17,10,348,0,348
1,18 – 29,37,1077,134,1211
2,30 – 49,59,1389,535,1924
3,50 – 69,22,733,304,1037
4,70+,3,205,34,239
5,Age Suppressed,6,—,—,6
6,Pending,0,0,0,0


In [11]:
table3 = TableToList(tables[2])
table3 = table3[1:]
#table3

In [12]:
cases_by_sex = ListToDf(table3, 5)
cases_by_sex.columns.values[0] = 'Sex'
cases_by_sex

,Sex,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,Female,56,1821,2,1823
1,Male,75,1902,992,2894
2,Unknown,0,29,13,42
3,Pending,6,0,0,6


In [13]:
table4 = TableToList(tables[3])
#table4

In [14]:
testing_status = ListToDf(table4, 2)
testing_status

,Testing Status,Total
0,Positive,4759
1,Negative,55997
2,Inconclusive,170
3,Total Tests,60926
4,Symptomatic Status,Total
5,Asymptomatic,113
6,Symptomatic,947
7,Pending,3699


In [15]:
table5 = TableToList(tables[4])
#table5

In [16]:
high_risk = ListToDf(table5[:4],2)
high_risk

,High Risk,Total
0,Health Care Workers (HCW),193


In [17]:
transmission_method = ListToDf(table5[4:], 2)
transmission_method

,Transmission Method,Total
0,Community Close Contact Transmission,817
1,Persons Incarcerated at Federal Prison in Lomp...,1007
2,Travel Transmission,12
3,Community Transmission,615
4,Under Investigation,2308


In [21]:
table6 = TableToList(tables[5])

table6

['RECOVERY STATUS',
 'Community ',
 'Federal Prison in Lompoc',
 'Total(Community & Prison)',
 'Active Cases',
 '394',
 '7',
 '401',
 'Recovered Cases',
 '3329',
 '997',
 '4326',
 'Recovering in Hospital',
 '—',
 '—',
 '—',
 'Recovering in ICU',
 '—',
 '—',
 '—',
 'Pending Information',
 '—',
 '—',
 '—']

In [22]:
recovery_status = ListToDf(table6, 4)
recovery_status

,RECOVERY STATUS,Community,Federal Prison in Lompoc,Total(Community & Prison)
0,Active Cases,394,7,401
1,Recovered Cases,3329,997,4326
2,Recovering in Hospital,—,—,—
3,Recovering in ICU,—,—,—
4,Pending Information,—,—,—


In [ ]:
table7 = TableToList(tables[6])
#table7

In [ ]:
ethnicity = ListToDf(table7, 5)
ethnicity